In [2]:
imdb.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16501718269104802468
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1547036262
locality {
  bus_id: 1
}
incarnation: 12708826497356550644
physical_device_desc: "device: 0, name: GeForce 930MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
import imdb
x_train, y_train = imdb.load_data(train=True)
x_test, y_test = imdb.load_data(train=False)

In [4]:
total_words=[]
for i in x_test+x_train:
    total_words.append(i)
num_words=list(set(total_words))

In [5]:
print(len(num_words)) 

49582


In [6]:
tokenizer=Tokenizer(num_words=None)
tokenizer.fit_on_texts(x_train+x_test)
x_train_token=tokenizer.texts_to_sequences(x_train)
x_test_token=tokenizer.texts_to_sequences(x_test)
max_tokens=600
x_train_pad = pad_sequences(x_train_token, maxlen=max_tokens,padding='pre', truncating='pre')
x_test_pad = pad_sequences(x_test_token, maxlen=max_tokens,padding='pre', truncating='pre')


In [7]:
model=Sequential()
model.add(Embedding(input_dim=len(num_words),output_dim=8,input_length=max_tokens,name='layer_embedding'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=16))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(1e-3),metrics=['accuracy'])

In [9]:

#Now lets train the model#Now let 
model.fit(x_train_pad,y_train,validation_split=0.05,epochs=1,batch_size=100)
#Performance 


Train on 23750 samples, validate on 1250 samples
Epoch 1/1
23750/23750 [==============================] - 802s - loss: 0.6849 - acc: 0.5453 - val_loss: 0.7664 - val_acc: 0.4184


In [12]:
from googletrans import Translator
tr=Translator()
text=tr.translate('안녕하세요.')
test=[text.text]
test_token=tokenizer.texts_to_sequences(test)
tokens_pad = pad_sequences(test_token, maxlen=600,padding='pre', truncating='pre')
model.predict(tokens_pad)



array([[ 0.54294997]], dtype=float32)

In [13]:
text=tr.translate('この文章は日本語で書かれました')
test=[text.text]
test_token=tokenizer.texts_to_sequences(test)
tokens_pad = pad_sequences(test_token, maxlen=600,padding='pre', truncating='pre')
model.predict(tokens_pad)

array([[ 0.641119]], dtype=float32)

In [14]:
text=tr.translate('Tiu frazo estas skribita en Esperanto.')
test=[text.text]
test_token=tokenizer.texts_to_sequences(test)
tokens_pad = pad_sequences(test_token, maxlen=600,padding='pre', truncating='pre')
model.predict(tokens_pad)

array([[ 0.68429518]], dtype=float32)